# ChatClovaX

This notebook provides a quick overview for getting started with Naver’s CLOVA [chat models](https://python.langchain.com/v0.2/docs/concepts/#chat-models) via CLOVA Studio. For detailed documentation of all ChatClovaX features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/chat_models/langchain_anthropic.chat_models.ChatAnthropic.html).

CLOVA Studio has several chat models. You can find information about latest models and their costs, context windows, and supported input types in the CLOVA Studio API Guide [documentation](https://api.ncloud-docs.com/docs/clovastudio-chatcompletions).

## Overview
### Integration details

| Class | Package | Local | Serializable | [JS support](https://js.langchain.com/v0.2/docs/integrations/chat/naver) | Package downloads | Package latest |
| :--- | :--- |:-----:| :---: |:------------------------------------------------------------------------:| :---: | :---: |
| [ChatClovaX](https://api.python.langchain.com/en/latest/chat_models/langchain_naver.chat_models.ChatClovaX.html) | [langchain-naver](https://api.python.langchain.com/en/latest/naver_api_reference.html) |   ❌   | beta/❌ |                                    ❌                                     | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-naver?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-naver?style=flat-square&label=%20) |

### Model features
| [Tool calling](/docs/how_to/tool_calling/) | [Structured output](/docs/how_to/structured_output/) | JSON mode | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
|:------------------------------------------:| :---: | :---: | :---: |  :---: | :---: |:-----------------------------------------------------:| :---: |:------------------------------------------------------:| :---: |
|❌| ✅/❌ | ✅/❌ | ❌ | ❌ | ❌ |                           ✅                           | ✅/❌ |                           ✅                            | ✅/❌ | 

## Setup

아래 절차에 대한 간략한 설명

- NCP 계정 생성 (https://www.ncloud.com/)
- CLOVA Studio 이용 신청 (https://www.ncloud.com/product/aiService/clovaStudio > ‘Get Started’)
- CLOVA Studio 테스트 앱 또는 서비스 앱 생성 후 API Key 확인 (참고: https://guide.ncloud-docs.com/docs/en/clovastudio-playground01#테스트앱생성)

### Credentials

- 현재 CLOVA Studio는 NCP_CLOVASTUDIO_API_KEY, NCP_APIGW_API_KEY 2개를 필요로 함
    - NCP_CLOVASTUDIO_API_KEY는 앱별로 다르다는 점에 유의 필요 (반면, NCP_APIGW_API_KEY는 계정당 하나)
- 두 API Key는 CLOVA Studio의 앱 신청 현항 (App Request Status) > 서비스 앱, 테스트 앱 목록 > 앱 별 ‘자세히 (Details)’ 버튼을 클릭하여 확인 가능
    - ref) https://clovastudio.ncloud.com/studio-application/service-app

- TODO: Update with relevant info.

Head to (TODO: link) to sign up to Naver and generate an API key. Once you've done this set the NAVER_API_KEY environment variable:

In [ ]:
import getpass
import os

os.environ["NAVER_API_KEY"] = getpass.getpass("Enter your Naver API key: ")

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### Installation

The LangChain Naver integration lives in the `langchain-naver` package:

In [ ]:
%pip install -qU langchain-naver

## Instantiation

- 예시 모델: HCX-DASH-001
- 파라미터: 템플릿 중 작동하는 항목 및 값으로 작성 (temperature, max_tokens, max_retries)
- 추가 파라미터: CLOVA Studio 특화 파라미터 작성


In [ ]:
from langchain_naver import ChatClovaX

llm = ChatClovaX(
    model="HCX-DASH-001",
    temperature=0.5,
    max_tokens=None,
    max_retries=2,
    # clovastudio_api_key="..."  # if you prefer to pass api key in directly instead of using env vars
    # task_id="..."  # if you want to use fine-tuned model
    # service_app=False (default)  # True if using service app
    # include_ai_filters=False (default) ## True if you want to detect inappropriate content
    # other params...
)

## Invocation

- 예시 프롬프트 및 작업

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Korean. Translate the user sentence.",
    ),
    ("human", "I love using AI of NAVER CLOVA X."),
]
ai_msg = llm.invoke(messages)
ai_msg

In [ ]:
print(ai_msg.content)

## Chaining

- 예시 프롬프트 및 작업


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}. Translate the user sentence.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Korean",
        "input": "I love using AI of NAVER CLOVA X.",
    }
)

## Additional functionalities

### Fine-tuning

You can call fine-tuned CLOVA X models by passing in your corresponding `task_id` parameter. (You don’t need to specify the model_name parameter when calling fine-tuned model.)

You can check `task_id` from corresponding test app/service app details.

In [ ]:
fine_tuned_model = ChatClovaX(
    task_id='abcd123e',
    temperature=0.5,
)

fine_tuned_model(messages)

### Service app

- 실 서비스 적용 시 서비스 앱 사용 필요 안내 (going live/in production)
  - ref) https://guide.ncloud-docs.com/docs/en/clovastudio-playground01#서비스앱신청
- 서비스 앱의 경우 별도 API Key가 발급되며 해당 API Key로만 호출 가능한 점 설명
- service_app 파라미터 사용 사례 추가

### AI Filter

- 간략한 AI Filter 소개 및 include_ai_filters 파라미터 사용 사례 추가
  - ref) https://guide.ncloud-docs.com/docs/en/clovastudio-playground01#AIFilter
- include output metadata with ai_filter

## API reference

For detailed documentation of all ChatClovaX features and configurations head to the API reference: https://api.python.langchain.com/en/latest/chat_models/langchain_naver.chat_models.ChatClovaX.html